In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r /content/drive/MyDrive/output_maps /content/

In [5]:
!cp /content/drive/MyDrive/image_functions.py /content/
!cp /content/drive/MyDrive/pipeline.py /content/

In [8]:
!cp /content/drive/MyDrive/laneline.py /content/

In [4]:
mkdir output_lane

Detección del Carril

In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pickle
import random
import glob

import laneline
from image_functions import *
from pipeline import *

from collections import deque
from moviepy.editor import VideoFileClip




path = "/content/output_maps/"


tst_imgs, tst_names = read_all_imgs(path)


print("Analizing " + str(len(tst_imgs)) + " images")






#New color mask 2
print("Applying Color Mask")
ncmask_2 = []
for img in tst_imgs:
    new_color_masked = new_color_mask_2(img)
    ncmask_2.append(new_color_masked)
print("Done")

print("Applying Direction Gradient")
direction = []
for img in ncmask_2:
    dir_img = dir_thresh(img, kernel_size = 7, thresh = (-0.3,0.3))
    direction.append(dir_img)
print("Done")

print("Applying Median Filter")
median_bin = []
for img in direction:
    median_img = MEDIAN_BLUR(img, kernel_size = 11)
    #median_img = MEDIAN_BLUR(median_img, kernel_size = 3)
    median_img = inverse(median_img)
    median_bin.append(median_img)
print("Done")

perspective = []
Minverse = []
print("Applying Perspective Transform")
for img in median_bin:
    perspective_img,M,Minv = perspective_transform(img)
    perspective.append(perspective_img)
    Minverse.append(Minv)
print("Done")

#Processed images
raw = []
left_raw = []
right_raw = []
print("Detecting Lines")
for img in perspective:
    left_fit_raw, right_fit_raw, out_img_raw = find_line_raw(img, return_img=True, plot_boxes=True, plot_line=True)
    raw.append(out_img_raw)
    left_raw.append(left_fit_raw)
    right_raw.append(right_fit_raw)
print("Done")


print("Processing Images")
processed = []
i = 0
for img in raw:
    print("Image: " + str(i))
    processed_img = plot_lanelines(tst_imgs[i],perspective[i],left_raw[i], right_raw[i], Minverse[i], img)
    mpimg.imsave('/content/output_lane/' + tst_names[i], processed_img)
    processed.append(processed_img)
    i += 1
print("Done")


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2703360/45929032 bytes (5.9%)5488640/45929032 bytes (12.0%)8421376/45929032 bytes (18.3%)11329536/45929032 bytes (24.7%)14499840/45929032 bytes (31.6%)17506304/45929032 bytes (38.1%)20570112/45929032 bytes (44.8%)23199744/45929032 bytes (50.5%)26255360/45929032 bytes (57.2%)29392896/45929032 bytes (64.0%)32497664/45929032 bytes (70.8%)35422208/45929032 bytes (77.1%)

Detección de Objetos

In [1]:
!pip install numpy==1.16.2
import numpy as np
print(np.__version__)

1.16.2


In [2]:
input_dir = '/content/output_lane'

In [56]:
!mkdir /content/output_objetos
!mkdir /content/output_objetos/detections_images
!mkdir /content/output_objetos/detections_txt

In [4]:
!git clone https://gitlab.com/victorvirgilio/yolov3-model.git

Cloning into 'yolov3-model'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 41 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [10]:
pip uninstall tensorflow

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.0


In [11]:
pip install tensorflow==1.6.0

     |████████████████████████████████| 45.9MB 104kB/s 
     |████████████████████████████████| 3.1MB 45.6MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107222 sha256=1e748471c3c26dcb98a6efb56adce6f842c637ec4240745354c118deaac8766b
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.1
    Uninstalling bleach-3.2.1:
      Successfully uninstalled bleach-3.2.1
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0


In [12]:
pip uninstall keras

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3


In [13]:
pip install keras==2.1.5

     |████████████████████████████████| 337kB 11.3MB/s 


In [62]:
%cd yolov3-model
!ls

/content/yolov3-model
detector  yolov3.h5


In [15]:
from detector.yolo import YOLO

import moviepy.editor as mpy
import time
import numpy as np

yolo = YOLO(model_path = 'yolov3.h5' , 
        anchors_path = 'detector/yolo_anchors.txt',
        classes_path = 'detector/coco_classes.txt',
        score = 0.8,
        iou = 0.7,
        model_image_size = (608, 608))

Using TensorFlow backend.


yolov3.h5 model, anchors, and classes loaded.


In [16]:
import cv2
import os
import glob
from PIL import Image

def draw_boxes(img, boxes):
    for box in boxes:
        cv2.rectangle(img, box[0], box[1], (0,0,255), 2)
    return Image.fromarray(np.uint8(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))

In [17]:
names = []
names = glob.glob('/content/output_lane/*')
print(names)       

['/content/output_lane/00006528.png', '/content/output_lane/00006905.png', '/content/output_lane/00006757.png', '/content/output_lane/00002566.png', '/content/output_lane/00003397.png', '/content/output_lane/00005419.png', '/content/output_lane/00008398.png', '/content/output_lane/00004430.png', '/content/output_lane/00005707.png', '/content/output_lane/00006000.png', '/content/output_lane/00005398.png', '/content/output_lane/00002211.png']


In [44]:
output_dir = '/content/output_objetos/'

In [63]:
import time
import glob
from keras.preprocessing import image

save = True

# names = glob.glob(input_dir + '*.png')[:5]
print('Cantidad de imágenes:   {0}\n\n'.format(len(names)))

k = 0
yolov3_detections = {}
times = []
print('\n***** ***** ***** Inicio de la detección ***** ***** *****\n')
for filename in names:

    img = image.load_img(filename)
    # Get time
    start = time.time()
    dimg, out_boxes, out_scores, out_classes = yolo.detect_image(img)
    end = time.time()
    times.append(end - start)

    #print("Detecto " + str(len(out_scores)) + " objetos")
    name = filename.split('/')[-1]
    name_f = name.split('.')[0]
    f = open(output_dir + "detections_txt/" + name_f + ".txt", "w")
    for i in range(len(out_classes)):
      if out_classes[i] == 0:
        #print('Person' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
        f.write('Person' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
      elif out_classes[i] == 1 or out_classes[i] == 2 or out_classes[i] == 3 or out_classes[i] == 5 or out_classes[i] == 7:
        #print('Car' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
        f.write('Car' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
      elif out_classes[i] == 9:
        #print('Traffic Light' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
        f.write('Traffic Light' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
      else:
        #print('Other' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
        f.write('Other' + ',' + str(int(out_boxes[i][0])) + ',' + str(int(out_boxes[i][1])) + ',' + str(int(out_boxes[i][2])) + ',' + str(int(out_boxes[i][3])) + ',' + str(out_scores[i]))
      f.write('\n')
    f.close()


    ouput_name = filename.split('/')
    name = ouput_name[-1]
    dimg.save(output_dir + 'detections_images/' + name)
    print('\n***** Imagen numero', k, 'fue procesada. *****')
    k+=1


tiempo_promedio = sum(times) / len(times)
print('\n\n\tPromedio de inferencia: {0}\n'.format(tiempo_promedio))

Cantidad de imágenes:   12



***** ***** ***** Inicio de la detección ***** ***** *****


***** Imagen numero 0 fue procesada. *****

***** Imagen numero 1 fue procesada. *****

***** Imagen numero 2 fue procesada. *****

***** Imagen numero 3 fue procesada. *****

***** Imagen numero 4 fue procesada. *****

***** Imagen numero 5 fue procesada. *****

***** Imagen numero 6 fue procesada. *****

***** Imagen numero 7 fue procesada. *****

***** Imagen numero 8 fue procesada. *****

***** Imagen numero 9 fue procesada. *****

***** Imagen numero 10 fue procesada. *****

***** Imagen numero 11 fue procesada. *****


	Promedio de inferencia: 7.412349800268809



Detección del Estado del Semáforo

In [71]:
%cd /content/

/content


In [75]:
mkdir output_tl

In [33]:
output_tl= '/content/output_tl/'

In [86]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pickle
import random
import glob
import csv

import laneline
from image_functions import *
from pipeline import *


def color_mask(img):
    """
    New color mask fucntion that combines three different color space filters
    to increasee detection accuracy.
    """

    hsv = HSV(img)
    hls = HLS(img)
    binary = np.zeros_like(img[:, :, 0])

    # Green filter
    #RGB
    lower_green = (90,235,70)
    upper_green = (210,255,170)
    rgb_g_mask = cv2.inRange(img, lower_green, upper_green)
    #HSV
    lower_green_hsv = (45, 80, 230)
    upper_green_hsv = (60, 180, 255)
    hsv_g_mask = cv2.inRange(hsv, lower_green_hsv, upper_green_hsv)
    #HLS
    lower_green_hls = (45,150,205)
    upper_green_hls = (60,220,255)
    hls_g_mask = cv2.inRange(hls, lower_green_hls, upper_green_hls)
    mask_g = cv2.bitwise_and(hsv_g_mask, hls_g_mask, rgb_g_mask)
    green_filter = binary | mask_g

    # Red filter
    #RGB
    lower_red = (240,150,75)
    upper_red = (255,180,95)
    rgb_r_mask = cv2.inRange(img, lower_red, upper_red)
    #HSV
    lower_red_hsv = (0, 155, 240)
    upper_red_hsv = (20, 195, 255)
    hsv_r_mask = cv2.inRange(hsv, lower_red_hsv, upper_red_hsv)
    #HLS
    lower_red_hls = (0,160,240)
    upper_red_hls = (20,190,255)
    hls_r_mask = cv2.inRange(hls, lower_red_hls, upper_red_hls)

    mask_r = cv2.bitwise_and(hsv_r_mask, hls_r_mask, rgb_r_mask)
    red_filter = binary | mask_r

    return green_filter, red_filter

def HLS(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

def HSV(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

def read_all_imgs(path, color=cv2.IMREAD_COLOR):

    images = []
    filenames = []

    filelist = os.listdir(path)
    for file in filelist:

        try:
            img = cv2.imread(path + file, color)
        except:
            img = None

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            filenames.append(file)

    return images, filenames

def read_all_txt(path):

    filenames = []

    filelist = os.listdir(path)
    for file in filelist:
        filenames.append(file)

    return filenames

def is_file_empty(file_path):
    """ Check if file is empty by confirming if its size is 0 bytes"""
    # Check if file exist and it is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0

def traffic_light_state(img, y_top, x_left, y_bottom, x_right):
    ROI = img[y_top:y_bottom, x_left:x_right]
    green, red = color_mask(ROI)
    nz_green = green.nonzero()
    nz_red = red.nonzero()
    nzg = nz_green[0]
    nzr = nz_red[0]
    if len(nzg) > len(nzr):
        return True
    else:
        return False

def all_trafic_light_state(img, y_top, x_left, y_bottom, x_right):
    if x_left > 300 and x_left < 1200:
        base = x_right - x_left
        height = y_bottom - y_top
        area = base * height
        if area > 140:
            ROI = img[y_top:y_bottom, x_left:x_right]
            green, red = color_mask(ROI)
            nz_green = green.nonzero()
            nz_red = red.nonzero()
            nzg = nz_green[0]
            nzr = nz_red[0]
            if len(nzg) > len(nzr):
                return 1
            else:
                return -1
        else:
            return 0
    else:
        return 0

print("Reading images...")
path_detected = "/content/output_objetos/detections_images/"
path_original = "/content/output_maps/"
path_txt = "/content/output_objetos/detections_txt/"


detected_images, detected_images_names = read_all_imgs(path_detected)
print(detected_images_names)
original_images, original_images_names = read_all_imgs(path_original)
print(original_images_names)
txt_names = read_all_txt(path_txt)
#print(txt_names)
names_txt = []
for name in original_images_names:
  name_f=name.split('.')[0]+'.txt'
  names_txt.append(name_f)
# names_txt.append(txt_names[8])
# names_txt.append(txt_names[3])
# names_txt.append(txt_names[0])
# names_txt.append(txt_names[9])
# names_txt.append(txt_names[1])
# names_txt.append(txt_names[7])
# names_txt.append(txt_names[10])
# names_txt.append(txt_names[4])
# names_txt.append(txt_names[3])
# names_txt.append(txt_names[5])
# names_txt.append(txt_names[11])
# names_txt.append(txt_names[2])
print(names_txt)

print("Analizing " + str(len(detected_images)) + " detected images")
print("Analizing " + str(len(original_images)) + " original images")
print("Analizing " + str(len(names_txt)) + " text files")

# for i in range(len(txt_names)):
#     name = original_images_names[i].split('/')[-1]
#     print("Imagen: " + name)
#     is_empty = is_file_empty(txt_names[i])
#     if is_empty:
#         print("Traffic Light Not Detected")
#         continue
#     else:
#         with open(txt_names[i], 'r') as file:
#             reader = csv.reader(file)
#             tl_flag = False
#             for row in reader:
#                 if row[0] == "Traffic Light":
#                     traffic_light_row = row
#                     print(traffic_light_row)
#                     tl_flag = True
#                     break
#             if tl_flag:
#                 if int(traffic_light_row[2]) > 300 and int(traffic_light_row[2]) < 1000:
#                     base = int(traffic_light_row[4]) - int(traffic_light_row[2])
#                     height = int(traffic_light_row[3]) - int(traffic_light_row[1])
#                     area = base * height
#                     if area < 140:
#                         print("Traffic Light Detected but too small to analyze")
#                     else:
#                         state = traffic_light_state(original_images[i],int(traffic_light_row[1]),int(traffic_light_row[2]),int(traffic_light_row[3]),int(traffic_light_row[4]))
#                         if state:
#                             print("Traffic Light Detected: Go")
#                         else:
#                             print("Traffic Light Detected: Stop")
#                 else:
#                     print("Traffic Light Detected but not in range")
#             else:
#                 print("Traffic Light Not Detected")


font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
thickness = 2

for i in range(len(names_txt)):
    name = detected_images_names[i].split('/')[-1]
    print("Imagen: " + name)
    is_empty = is_file_empty(path_txt + names_txt[i])
    if is_empty:
        print("Traffic Light Not Detected")
        new_image = cv2.putText(detected_images[i], 'Traffic Light Not Detected', org, font, fontScale, (255,255,255), thickness, cv2.LINE_AA)
    else:
        with open(path_txt + names_txt[i], 'r') as file:
            reader = csv.reader(file)
            traffic_light_row = []
            tl_flag = False
            for row in reader:
                if row[0] == "Traffic Light":
                    tl_flag = True
                    traffic_light_row.append(row)
            if tl_flag:
                sum_state = 0
                for tl in traffic_light_row:
                    state = all_trafic_light_state(original_images[i],int(tl[1]),int(tl[2]),int(tl[3]),int(tl[4]))
                    #print(state)
                    sum_state += state
                if sum_state == 0:
                    print("Traffic Light detected but too small to analyze or not in range")
                    new_image = cv2.putText(detected_images[i], 'Traffic Light Detected but too small to analyze or not in range', org, font, fontScale, (255,255,255), thickness, cv2.LINE_AA)
                elif sum_state > 0:
                    print("Traffic Light Detected: Go")
                    new_image = cv2.putText(detected_images[i], 'Traffic Light Detected: Go', org, font, fontScale, (0,255,0), thickness, cv2.LINE_AA)
                else:
                    print("Traffic Light Detected: Stop")
                    new_image = cv2.putText(detected_images[i], 'Traffic Light Detected: Stop', org, font, fontScale, (255,0,0), thickness, cv2.LINE_AA)
            else:
                print("Traffic Light Not Detected")
                new_image = cv2.putText(detected_images[i], 'Traffic Light Not Detected', org, font, fontScale, (255,255,255), thickness, cv2.LINE_AA)
    mpimg.imsave(output_tl + name, new_image )
    print(name)

Reading images...
['00006528.png', '00006905.png', '00006757.png', '00002566.png', '00003397.png', '00005419.png', '00008398.png', '00004430.png', '00005707.png', '00006000.png', '00005398.png', '00002211.png']
['00006528.png', '00006905.png', '00006757.png', '00002566.png', '00003397.png', '00005419.png', '00008398.png', '00004430.png', '00005707.png', '00006000.png', '00005398.png', '00002211.png']
['00006528.txt', '00006905.txt', '00006757.txt', '00002566.txt', '00003397.txt', '00005419.txt', '00008398.txt', '00004430.txt', '00005707.txt', '00006000.txt', '00005398.txt', '00002211.txt']
Analizing 12 detected images
Analizing 12 original images
Analizing 12 text files
Imagen: 00006528.png
Traffic Light Detected: Stop
00006528.png
Imagen: 00006905.png
Traffic Light Detected: Stop
00006905.png
Imagen: 00006757.png
Traffic Light Not Detected
00006757.png
Imagen: 00002566.png
Traffic Light Not Detected
00002566.png
Imagen: 00003397.png
Traffic Light Not Detected
00003397.png
Imagen: 0000